In [1]:
using Pkg
Pkg.activate(joinpath(@__DIR__))

  Activating project at `~/Masterarbeit/debug_pik_cluster`


In [2]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 16 × Intel(R) Xeon(R) CPU E5-2667 v3 @ 3.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, haswell)
  Threads: 1 on 16 virtual cores
Environment:
  LD_LIBRARY_PATH = /p/system/packages/julia/1.7.0/lib
  JULIA_ROOT = /p/system/packages/julia/1.7.0


In [3]:
using MKL
using LinearAlgebra
println("BLAS:")
println("  num_threads: ", BLAS.get_num_threads())
println("  config: ", BLAS.get_config())

BLAS:
  num_threads: 1
  config: LBTConfig([ILP64] libmkl_rt.so, [LP64] libmkl_rt.so)


In [4]:
using OrdinaryDiffEq
using BenchmarkTools

In [5]:
#number of ODE equations
N = 100
println("Number of ODE equations: ", N)

Number of ODE equations: 100


In [6]:
#simple ODE problem (https://de.mathworks.com/company/newsletters/articles/stiff-differential-equations.html)
u = zeros(N)
du = zeros(N)

delta = 0.002
sim_time = 2.0/delta

u = u .+ delta

function foo(du, u, p, t)
    for i in 1:N
        du[i] = (u[i]^2 - u[i]^3)
    end
end

foo (generic function with 1 method)

In [7]:
prob = ODEProblem(foo, u, (0.0, sim_time), nothing)
println()

In [8]:
#check how long the solver needs
@benchmark solve(prob, Rodas4())

BenchmarkTools.Trial: 47 samples with 1 evaluation.
 Range (min … max):   89.904 ms … 121.689 ms  ┊ GC (min … max):  0.00% … 11.05%
 Time  (median):     104.938 ms               ┊ GC (median):    12.34%
 Time  (mean ± σ):   106.574 ms ±   8.795 ms  ┊ GC (mean ± σ):   7.59% ±  5.85%

                          ▂   ▅                            █     
  ▅▁▁▅▅▁▅▅▁▅▁▁▁▁▅▁▅▁▅▅▅▁████▅▁█▁▁██▅▅▁▁▅▁▁▁▅▁▁█▁▅▅▅▁▁▁▁▁▅▅▁█▁▁█ ▁
  89.9 ms          Histogram: frequency by time          122 ms <

 Memory estimate: 77.68 MiB, allocs estimate: 2620793.

In [9]:
@time sol = solve(prob, Rodas4())
println()

  0.114929 seconds (2.62 M allocations: 77.680 MiB, 27.17% gc time)



In [10]:
size(sol.t)

(41,)

In [11]:
sol.u[end][1]

0.9999999999978489